In [ ]:
%matplotlib widget
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import IPython
import numpy as np
from ipywidgets import interact

In [ ]:
rate, wav = wavfile.read("dataset/Elesdi_se_vraca_kuci_mono.wav")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(wav, Fs=rate)
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=wav, rate=rate)

In [ ]:
rate, noised_wav = wavfile.read("dataset/Elesdi_se_vraca_kuci_mono_noised.wav")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(noised_wav, Fs=rate)
ax2.magnitude_spectrum(noised_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=noised_wav, rate=rate)

In [ ]:
plt.figure()
plt.magnitude_spectrum(noised_wav[:int(3*rate)], Fs=rate, scale='dB');
plt.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
plt.ylim(-120, 70);
plt.grid();

# Filtri
Sistemi za slabljenje odredjenih delova frekvencijskog spektra

# Tipovi Filtara
Osnovni filtri su: 

Low-Pass (LP; Niskofrekventni filtar) - Propušta sve sinusoide sa frekvencijom koja je manja od neke granične frekvencije.

High-Pass (HP; Visokofrekventni filtar) - Propušta sve sinusoide sa frekvencijom koja je veća od neke granične frekvencije.

Band-Pass (BP; Filtar propusnik opsega) - Propušta sve sinusoide sa frekvencijom koja je između neke dve granične frekvencije.

Band-Stop (BS; Filtar nepropusnik opsega) - Propušta sve sinusoide sa frekvencijom koja nije između neke dve granične frekvencije.

<center><img src="dataset/images/filterTypes.png"/></center>

Da bi smo znali kakav filtar tražimo, moramo znati šta on treba da radi.
Pošto filtri nisu idealni, oni ne mogu savršeno propuštati u propusnoj zoni i savršeno nepropuštati u nepropusnoj zoni. Takodje, da bi prešli iz jedne zone u drugu, oni moraju imati i prelaznu zonu.

Ovo sve se definiše gabaritima filtra, koji predstavljaju ekstremne slučajeve u frekventnoj karakteristici filtra.
Na primer, za Low-Pass filtar, treba definisati najveće slabljenje u propusnom opsegu i najmanje slabljenje u nepropusnom opsegu.

<center><img src="dataset/images/specifikacija.png"/></center>


# IIR Filtri

Imaju odziv koji polako opada ali nikad ne dodje do nule. Dobijaju se digitalizacijom analognih filtara.
Kako postoje više tipova analognih filtara, takozvane aproksimacije, postoji i više tipova IIR filtara, od kojih svaka ima različite karakteristike. Bitno je napomenuti da moraju da imaju frekvenciju odabiranja, jer su po prirodi diskretni. Za frekvenciju odabiranja uzimamo istu frekvenciju iz naše pesme.

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex = "all", figsize=(10, 8), sharey="all")
fig.subplots_adjust(hspace=0.2)

N = 5 #Red filtra
Wn = 4000 #Granična frekvencija
rp = 1 #Maksimalna ateunacija u propusnom opsegu
rs = 10 #Minimalna atenuacija u nepropusnom opsegu
    
butterB, butterA = signal.butter(N, Wn, fs=rate)
w, h = signal.freqz(butterB, butterA, fs = rate)
ax1.semilogx(w, 20*np.log10(h))

cheby1B, cheby1A = signal.cheby1(N, rp, Wn, fs=rate)
w, h = signal.freqz(cheby1B, cheby1A, fs = rate)
ax2.semilogx(w, 20*np.log10(h))

cheby2B, cheby2A = signal.cheby2(N, rs, Wn, fs=rate)
w, h = signal.freqz(cheby2B, cheby2A, fs = rate)
ax3.semilogx(w, 20*np.log10(h))

ellipB, ellipA = signal.ellip(N, rp, rs, Wn, fs=rate)
w, h = signal.freqz(ellipB, ellipA, fs = rate)
ax4.semilogx(w, 20*np.log10(h))

ax1.set_ylim(-40, 10)

ax1.axvline(Wn, color='green')
ax2.axvline(Wn, color='green')
ax3.axvline(Wn, color='green')
ax4.axvline(Wn, color='green')

ax2.axhline(-rp, color='green', ls='--')
ax3.axhline(-rs, color='green', ls='--')
ax4.axhline(-rp, color='green', ls='--')
ax4.axhline(-rs, color='green', ls='--')

ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()


U prethodnom kodu, najbitniji parametri za razmatranje su N i Wn, gde je N red filtra, gde veći redovi fitara predstavljaju "jače" filtre, koji su, u zavisnosti od aproksimacije, sa bržim opadanjem, manjom prelaznom zonom i slično. Za butterworthovu aproksimaciju na primer, to predstavlja brzinu opadanja van propusne zone. Wn predstavlja kraj propusne zone, tj graničnu frekvenciju.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

Wn = 4000

for N in range(1, 10, 2):
    b, a = signal.butter(N, Wn, fs=rate)
    w, h = signal.freqz(b, a, fs=rate)
    ax.semilogx(w, 20*np.log10(h), label = f"{N}")


ax.set_ylim(-40, 10)

ax.legend()
ax.grid()

Hajde da probamo da filtriramo našu pesmu batervortovim filtrom 5-og reda, sa graničnom frekvencijom od 4kHz. To je filtar koji smo napravili iznad u uporedjivanju različitih aproksimacija.

In [ ]:
upitno_filtriran_wav = signal.lfilter(butterB, butterA, noised_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(upitno_filtriran_wav, Fs=rate)
ax2.magnitude_spectrum(upitno_filtriran_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=upitno_filtriran_wav, rate=rate)

Sa grafika, a i sluhom takođe se može utvrditi da ovo rešenje nije optimalno. Svi visoki tonovi su ubijeni, a pištanje na 5kHz je i dalje postojano. Stoga nam je potrebno bolje rešenje. Za početak ćemo probati samo da ubijemo pištanje na 5kHz.

In [ ]:
bandstop5kHz = signal.butter(5, (4900, 5100), 'bandstop', fs=rate)
malo_bolje_filtriran_wav = signal.lfilter(bandstop5kHz[0], bandstop5kHz[1], noised_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(malo_bolje_filtriran_wav, Fs=rate)
ax2.magnitude_spectrum(malo_bolje_filtriran_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=malo_bolje_filtriran_wav, rate=rate)

Može se primetiti da je sve na 5kHz potpuno utišano. Takodje nam ostaje šum izmedju 8 i 10 kHz. To je sledeće potrebno filtrirati.

In [ ]:
bandstop8kHz12kHz = signal.ellip(12, 0.1, 70, (7900, 12100), 'bandstop', fs=rate)
dosta_bolje_filtriran_wav = signal.lfilter(bandstop8kHz12kHz[0], bandstop8kHz12kHz[1], malo_bolje_filtriran_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(dosta_bolje_filtriran_wav, Fs=rate)
ax2.magnitude_spectrum(dosta_bolje_filtriran_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=dosta_bolje_filtriran_wav, rate=rate)

# FIR Filtri

Imaju odziv konstantne dužine. Dobijaju se direktnim projektovanjem u digitalnom domenu. Prednost toga je što mogu da budu i stepeničastog oblika, tj da imaju različite gabarite za različite opsege. 

<center><img src="dataset/images/stepenice.svg" /></center>

Mnogo su jednostavniji za implementaciju od IIR filtara, uvek su stabilni i imaju linearnu fazu (konstantno vremensko kašnjenje).

Umesto reda je bitan broj odbiraka, i što je on veći, filtar može da radi ekstremnije stvari, tj može da ima veća slabljenja u nepropusnim zonama, manje prelazne zone i slično. To naravno dovodi i do sporijeg računanja i veće kompkleksnosti, stoga se treba truditi da taj broj bude minimalan moguć.

Prvo ćemo napraviti Low-pass filter na 4kHz.

In [ ]:
fir4kHz = signal.firls(199, (0, 4000, 4100, rate//2), (1, 1, 0, 0), fs=rate)

In [ ]:
plt.figure()

w, h = signal.freqz(fir4kHz)

plt.semilogx(0.5*rate*w/np.pi, np.real(20*np.log10(h)))

plt.grid()

Hajde da takođe probamo da projektujemo Band-pass za 5kHz.


In [ ]:
fir5kHz = signal.firls(199, (0, 4900, 4950, 5050, 5100, rate//2), (1, 1, 0, 0, 1, 1), fs=rate)

In [ ]:
plt.figure()

w, h = signal.freqz(fir5kHz)

plt.semilogx(0.5*rate*w/np.pi, np.real(20*np.log10(h)))

plt.grid()

Ukoliko nismo zadovoljni dobijenim filterom, možemo povećavati brojeve odbiraka.

In [ ]:
fir5kHz_longer = signal.firls(699, (0, 4850, 4900, 5100, 5150, rate//2), (1, 1, 0, 0, 1, 1), fs=rate)

In [ ]:
plt.figure()

w, h = signal.freqz(fir5kHz_longer)

plt.semilogx(0.5*rate*w/np.pi, np.real(20*np.log10(h)))

plt.grid()

Hajde da primenimo ovaj filtar na nas zašumljen signal.

In [ ]:
fir5kHz_wav = signal.lfilter(fir5kHz_longer, [1.0], noised_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(fir5kHz_wav, Fs=rate)
ax2.magnitude_spectrum(fir5kHz_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=fir5kHz_wav, rate=rate)

Očigledno je da je filter dobro izbacio pištanje na 5kHz.
Takodje je potrebno napraviti Band-pass od 8kHz do 12kHz.

In [ ]:
fir8kHz12kHz = signal.firls(699, (0, 7900, 8000, 12000, 12100, rate//2), (1, 1, 0, 0, 1, 1), fs=rate)

In [ ]:
plt.figure()

w, h = signal.freqz(fir8kHz12kHz)

plt.semilogx(0.5*rate*w/np.pi, np.real(20*np.log10(h)))

plt.grid()

In [ ]:
fir_filtered_wav = signal.lfilter(fir8kHz12kHz, [1.0], fir5kHz_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(fir_filtered_wav, Fs=rate)
ax2.magnitude_spectrum(fir_filtered_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=fir_filtered_wav, rate=rate)

Takođe je moguće napraviti objedinjen filter ovom metodom.

In [ ]:
fir_combined = signal.firls(699, (0, 4800, 4900, 5100, 5200, 7800, 7900, 12100, 12200, rate//2), (1, 1, 0, 0, 1, 1, 0, 0, 1, 1), fs=rate)

In [ ]:
plt.figure()

w, h = signal.freqz(fir_combined)

plt.semilogx(0.5*rate*w/np.pi, np.real(20*np.log10(h)))

plt.grid()

In [ ]:
fir_filtered_combined_wav = signal.lfilter(fir_combined, [1.0], noised_wav)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.specgram(fir_filtered_combined_wav, Fs=rate)
ax2.magnitude_spectrum(fir_filtered_combined_wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.magnitude_spectrum(wav[:int(3*rate)], Fs=rate, scale='dB');
ax2.set_ylim(-120, 70);
ax2.grid();
IPython.display.Audio(data=fir_filtered_combined_wav, rate=rate)

# Implementacije

Direktna Implementacija FIR filtra

<center><img src="dataset/images/directFir.png"/></center>

Kanonička Implementacija IIR filtra

<center><img src="dataset/images/CanonicalIIR.png"/></center>